# Credentials

__Put Testnet or Prod credentials here__:
(Testnet here: https://testnet.binancefuture.com/en/futures/BTCUSDT)

Testnet resets each day, but your credentials wont change.

In [4]:
api_key = "6ce63f3406fd8ebbff01054a66c25fe3c851c45932088c8ca3131a7005188462"
secret_key = "aa3ea32929252467fa5ffeac5818c95beabfb5dba691ef445e7eaa31ea0d15f6"

# Python Api Wrapper

It is important to install python wrappers of binance API (one is not made by Binance, use at your own risk), just run on terminal:

__pip install python-binance__

__pip install binance-futures-connector__

Currently unofficial wrapper has problems retrieving real-time prices (problem with websockets), so we use the official Api wrapper of Binance to this task.

Useful links:

__Binance Official Api Docs__: https://binance-docs.github.io/apidocs/futures/en/#account-trades-endpoints

__Unofficial Python Api Wrapper__: https://python-binance.readthedocs.io/en/latest/overview.html

__Official Binance Api Wrapper__: https://github.com/binance/binance-futures-connector-python 

Api Wrapper sometimes its not specific on arguments for functions, so it´s useful to check that args on official Api docs

__This is the code for live streaming that is going to be replaced with official api__

In [5]:
from binance import ThreadedWebsocketManager
import time
twm = ThreadedWebsocketManager(testnet = True)
twm.start()
twm.start_kline_futures_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")
time.sleep(10)
twm.stop()

'btcusdt_perpetual@continuousKline_1m'

CANCEL read_loop
Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ThreadedApiManager.start_listener() done, defined at /home/mauricio/anaconda3/lib/python3.9/site-packages/binance/threaded_stream.py:42> exception=RuntimeError("Task <Task pending name='Task-14' coro=<Queue.get() running at /home/mauricio/anaconda3/lib/python3.9/asyncio/queues.py:166> cb=[_release_waiter(<Future pendi...112a0c370>()]>)() at /home/mauricio/anaconda3/lib/python3.9/asyncio/tasks.py:416]> got Future <Future pending> attached to a different loop")>
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/threaded_stream.py", line 46, in start_listener
    msg = await asyncio.wait_for(s.recv(), 3)
  File "/home/mauricio/anaconda3/lib/python3.9/asyncio/tasks.py", line 479, in wait_for
    return fut.result()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/streams.py", line 197, in recv
    res = await asyncio.wait_f

__Useful code for streaming__

In [1]:
import time
#import logging
#from binance.lib.utils import config_logging
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient

#config_logging(logging, logging.DEBUG)


def message_handler(message):
    print(message)


my_client = CMFuturesWebsocketClient()
my_client.start()

my_client.kline(
    symbol="btcusd_perp",
    id=12,
    interval="1m",
    callback=message_handler,
)

time.sleep(10)

#logging.debug("closing ws connection")
my_client.stop()

{'id': 12, 'result': None}
{'e': 'kline', 'E': 1660163725943, 's': 'BTCUSD_PERP', 'k': {'t': 1660163700000, 'T': 1660163759999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503333722, 'L': 503333888, 'o': '23733.3', 'c': '23736.7', 'h': '23736.7', 'l': '23729.6', 'v': '6253', 'n': 167, 'x': False, 'q': '26.34858933', 'V': '3303', 'Q': '13.91798633', 'B': '0'}}
{'e': 'kline', 'E': 1660163728294, 's': 'BTCUSD_PERP', 'k': {'t': 1660163700000, 'T': 1660163759999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503333722, 'L': 503333889, 'o': '23733.3', 'c': '23736.8', 'h': '23736.8', 'l': '23729.6', 'v': '6254', 'n': 168, 'x': False, 'q': '26.35280220', 'V': '3304', 'Q': '13.92219920', 'B': '0'}}
{'e': 'kline', 'E': 1660163730635, 's': 'BTCUSD_PERP', 'k': {'t': 1660163700000, 'T': 1660163759999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503333722, 'L': 503333891, 'o': '23733.3', 'c': '23736.8', 'h': '23736.8', 'l': '23729.6', 'v': '6414', 'n': 170, 'x': False, 'q': '27.02686105', 'V': '3464', 'Q': '14.59625805', 'B'

# Streaming data

This is a simple class for streaming candles. It adds bars to a dataframe on a specified time.

__Important__: You maybe need to restart kernel each time you close a connection.

In [1]:
from binance.client import Client
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

In [2]:
class FuturesTrader():
    def __init__(self, interval = "1m"):
        self.stream = None
        self.data = pd.DataFrame(columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Complete"])
        self.data["Date"] = pd.to_datetime(self.data.iloc[:,0], unit = "ms")
        self.data.set_index("Date", inplace = True)
        self.interval = interval
        
    def message_handler(self, message):
        if 'result' in message.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(message["E"], unit = "ms")
        start_time = pd.to_datetime(message["k"]["t"], unit = "ms")
        first   = float(message["k"]["o"])
        high    = float(message["k"]["h"])
        low     = float(message["k"]["l"])
        close   = float(message["k"]["c"])
        volume  = float(message["k"]["v"])
        complete=       message["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
    def start_streaming(self):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol="btcusd_perp",
            id=12,
            interval=self.interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()

In [3]:
trader = FuturesTrader()

In [4]:
trader.start_streaming()

.............................................................................................................................................................................................................................................................................................................................................

In [5]:
trader.stop_streaming()

In [6]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-08-10 21:42:00,23931.1,23931.1,23871.4,23908.6,37001.0,True
2022-08-10 21:43:00,23908.9,23932.5,23908.9,23932.5,4748.0,True
2022-08-10 21:44:00,23932.4,23932.4,23896.2,23897.6,37548.0,True
2022-08-10 21:45:00,23897.6,23934.2,23897.6,23928.9,32553.0,True
2022-08-10 21:46:00,23928.8,23943.4,23925.0,23925.0,18477.0,True
2022-08-10 21:47:00,23925.0,23946.3,23922.6,23939.1,19230.0,True
2022-08-10 21:48:00,23939.2,23941.8,23932.8,23935.7,11189.0,True
2022-08-10 21:49:00,23935.7,23937.5,23935.7,23937.5,1452.0,True
2022-08-10 21:50:00,23937.5,23937.5,23905.5,23905.5,5986.0,False


# Retrieving historical data

When running the bot trader, we need the latest historical data to process all the needed metrics in real time. This method retrieves the latest __n__ candlesticks data on the required interval.

This lines are useful for saving candlesticks info for Neural Networks

params info at:  https://binance-docs.github.io/apidocs/futures/en/#kline-candlestick-data

In [39]:
from binance.client import Client
import pandas as pd

#### VARIABLES ####
max_number_of_candles = 10000 
interval_of_candles = "1m"
symbol = "BTCUSDT"
#### Available intervals for candles ####
time_to_past = {
 "1m"  : timedelta(minutes=max_number_of_candles),
 "3m"  : timedelta(minutes=max_number_of_candles*3),
 "5m"  : timedelta(minutes=max_number_of_candles*5),
 "15m" : timedelta(minutes=max_number_of_candles*15),
 "30m" : timedelta(minutes=max_number_of_candles*30),
 "1h"  : timedelta(hours=max_number_of_candles),
 "2h"  : timedelta(hours=max_number_of_candles*2),
 "4h"  : timedelta(hours=max_number_of_candles*4),
 "6h"  : timedelta(hours=max_number_of_candles*6),
 "8h"  : timedelta(hours=max_number_of_candles*8),
 "12h" : timedelta(hours=max_number_of_candles*12),
 "1d"  : timedelta(days=max_number_of_candles),
 "3d"  : timedelta(days=max_number_of_candles*3),
 "1w"  : timedelta(days=max_number_of_candles*7),
 "1M"  : timedelta(days=max_number_of_candles*28) #this may give less than the desired candles because each month has different amount of days
}
########################################

client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True) # Testnet!!!

now = datetime.utcnow()
past = str(now - time_to_past[interval_of_candles])

bars = client.futures_historical_klines(symbol = symbol, 
                                        interval = interval_of_candles, 
                                        start_str =past,
                                        end_str = None)
df = pd.DataFrame(bars)
df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
              "Close Time", "Quote Asset Volume", "Number of Trades",
              "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
df.set_index("Date", inplace = True)
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors = "coerce")
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-04 05:50:00,23780.0,23780.0,23198.0,23198.0,84.921
2022-08-04 05:51:00,23228.2,23900.0,23198.0,23900.0,84.941
2022-08-04 05:52:00,23852.2,23898.0,23198.0,23898.0,132.174
2022-08-04 05:53:00,23898.0,23900.0,23198.0,23228.2,74.257
2022-08-04 05:54:00,23520.4,23946.0,23198.0,23653.4,83.717
...,...,...,...,...,...
2022-08-11 04:26:00,24344.5,24344.5,24200.0,24317.6,75.895
2022-08-11 04:27:00,24312.2,24317.6,24150.0,24308.0,99.008
2022-08-11 04:28:00,24310.1,24313.6,24305.6,24311.0,21.804


Then we can save the data to prevent requesting too many times from futures api

In [42]:
df.to_csv("out.csv")

You can load the data running:

In [62]:
loaded = pd.read_csv('out.csv')
loaded.set_index("Date", inplace = True)
loaded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10001 entries, 2022-08-04 05:50:00 to 2022-08-11 04:30:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    10001 non-null  float64
 1   High    10001 non-null  float64
 2   Low     10001 non-null  float64
 3   Close   10001 non-null  float64
 4   Volume  10001 non-null  float64
dtypes: float64(5)
memory usage: 468.8+ KB
